# CNN ADAboost.M1



In [1]:
import os,gc,time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.backend import clear_session
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.base import BaseEstimator

RANDOM_SEED = 2023 # random seed for k-fold cross-validation
BATCH_SIZE = 16  
EPOCH_NUM = 50
FOLD_NUM = 5
num_channels = 3 # number of channels = 3: RGB
image_size = (112, 112) # set image size

# early stop metrics monitoring validation loss, in order to compare models, set all epoch=50 fixed, not used now.
#early_stopping_loss = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#early_stopping_accuracy = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
#CALLBACKS = [early_stopping_loss, early_stopping_accuracy]

# set CUDA environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # use GPU0
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" # reduce potential conflict of packages
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true" # GPU memory management

# set data dir
data_dir = "./data/merged_aug_data_224"
#train_dir = "./data/aug_data_224/Train"
#test_dir = "./data/aug_data_224/Test"

# set folder address, avoid duplicated directory name
model_weights_dir = './models/CNN_boost'
model_id = 0
while os.path.exists(model_weights_dir):
    model_id += 1
    model_weights_dir = f'./models/CNN_boost{model_id}/'
if not os.path.exists(model_weights_dir):
    os.makedirs(model_weights_dir)

# save all configurations in a txt file
with open(os.path.join(model_weights_dir, 'info.txt'), 'w') as file:
    variables = {
        'RANDOM_SEED': RANDOM_SEED,
        'BATCH_SIZE': BATCH_SIZE,
        'EPOCH_NUM': EPOCH_NUM,
        'FOLD_NUM': FOLD_NUM,
        'num_channels': num_channels,
        'image_size': image_size,
        #'monitor of early_stopping_loss':early_stopping_loss.monitor,
        #'patience of early_stopping_loss':early_stopping_loss.patience,
        #'restore_best_weights of early_stopping_loss':early_stopping_loss.restore_best_weights,
        #'monitor of early_stopping_accuracy': early_stopping_loss.monitor,
        #'patience of early_stopping_accuracy': early_stopping_loss.patience,
        #'restore_best_weights of early_stopping_accuracy': early_stopping_loss.restore_best_weights,
        'CUDA_VISIBLE_DEVICES': os.environ["CUDA_VISIBLE_DEVICES"],
        'KMP_DUPLICATE_LIB_OK': os.environ["KMP_DUPLICATE_LIB_OK"],
        'TF_FORCE_GPU_ALLOW_GROWTH': os.environ['TF_FORCE_GPU_ALLOW_GROWTH'],
        'data_dir:' : data_dir
        #'train_dir': train_dir,
        #'test_dir': test_dir
    }
    for variable, value in variables.items():
        file.write(f'{variable} = {value}\n')
print('Model weights have been saved at：', model_weights_dir)

C:\DevTools\Anaconda3\envs\tf\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Model weights have been saved at： ./models/CNN_boost


# Design the weak model 2

# Loading datasets

In [2]:
#Load training set and test set, and their labels
def load_images_and_labels(directory):
    images = []
    labels = []
    class_names = os.listdir(directory)
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        print(f'loading images from {class_dir}')
        for filename in tqdm(os.listdir(class_dir)):
            image_path = os.path.join(class_dir, filename)
            try:
                image = cv2.imread(image_path)
                #image = tf.cast(image, tf.float16) / 255.0 # normalize, out of memeory
                image = cv2.resize(image, image_size)
                images.append(image)
                labels.append(class_names.index(class_name))
            except Exception as e:
                print(f"Error loading image: {image_path}")
                print(f"Error message: {e}")
    return np.array(images), np.array(labels)

In [3]:
# Get training set and test set, and their label
images, labels = load_images_and_labels(data_dir)

loading images from ./data/merged_aug_data_224\Mild


100%|██████████████████████████████████████████████████████████████████████████| 11200/11200 [00:07<00:00, 1452.43it/s]


loading images from ./data/merged_aug_data_224\Moderate


100%|██████████████████████████████████████████████████████████████████████████| 16372/16372 [00:11<00:00, 1431.47it/s]


loading images from ./data/merged_aug_data_224\Non


100%|██████████████████████████████████████████████████████████████████████████| 12800/12800 [00:08<00:00, 1464.98it/s]


In [4]:
import numpy as np
import gc
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf

# create a base CNN model
def create_model():
    model = Sequential()
    model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(image_size[0], image_size[1], num_channels)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.20))
    
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model


In [5]:
def boost_cnn(n_iterations, train_images, train_labels, val_X=None, val_y=None):
    # initialize the sample weights
    num_samples = len(train_images)
    sample_weights = np.ones(num_samples) / num_samples
    
    # initialize the list of classifiers and their weights alpha
    classifiers = []
    alphas = []

    #iteration for each base estimator
    for i in range(n_iterations):
        print(f'No.{i} iteration:')
        # create and train the base estimator
        base_classifier = create_model()
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=2)
        base_classifier.fit(train_images, train_labels, 
                            sample_weight=sample_weights, 
                            epochs=EPOCH_NUM, 
                            batch_size=16, 
                            validation_data=(val_X, val_y),
                            callbacks=[early_stopping],
                            verbose=1)
        
        # use the trained base estimator to make predictions 
        predictions = base_classifier.predict(train_images)
        
        # update the error for each sample from training set
        #weighted_error = np.sum(sample_weights * (train_labels != predictions.argmax(axis=1))) / np.sum(sample_weights)
        weighted_error = np.sum(sample_weights * np.not_equal(train_labels, predictions.argmax(axis=1))) / np.sum(sample_weights)
        
        # calculate the weight for each base estimator alpha
        #alpha = (1 - weighted_error) / max(weighted_error, 1e-20)
        alpha = np.log((1 - weighted_error) / max(weighted_error, 1e-20))
        
        # update the sample weights
        #sample_weights *= np.exp(-alpha * (labels != predictions.argmax(axis=1)))
        sample_weights *= np.exp(-alpha * np.not_equal(train_labels, predictions.argmax(axis=1)))
        sample_weights /= np.sum(sample_weights)
    
        # save the classifiers and their weights alpha
        classifiers.append(base_classifier)
        alphas.append(alpha)
        #normalization of values of alphas, AUC score needs the sum of proba equals to 1
        sum_alpha = sum(alphas)
        alphas = [alpha / sum_alpha for alpha in alphas]
    return classifiers, alphas

# use the adaboost classifier to predict
def boost_predict(test_images, classifiers, alphas):
    # create to store the proba distribution of prediciton of each image
    test_probabilities = np.zeros((len(test_images), 3))
    print(sum(alphas))
    for i in range(len(classifiers)):
        # get the weighted predictions from each classifier
        pred_each_clf = classifiers[i].predict(test_images)
        #print(pred_each_clf[0:3])
        test_probabilities += alphas[i] * pred_each_clf
        test_predictions = np.argmax(test_probabilities, axis=1)
        #test_probabilities = tf.nn.softmax(test_probabilities)        
    
    return test_probabilities, test_predictions

def boost_test(test_labels, test_probabilities, test_predictions):
    # calculate metrics
    accuracy = accuracy_score(test_labels, test_predictions)
    f1 = f1_score(test_labels, test_predictions, average='weighted')
    precision = precision_score(test_labels, test_predictions, average='weighted')
    recall = recall_score(test_labels, test_predictions, average='weighted')
    confusion_mat = confusion_matrix(test_labels, test_predictions)
    weighted_auc_ovr = roc_auc_score(test_labels, test_probabilities, average='weighted',multi_class='ovr')
    #print(weighted_auc_ovr)
    weighted_auc_ovo = roc_auc_score(test_labels, test_probabilities, average='weighted',multi_class='ovo')
    #print(weighted_auc_ovo)
    print(f'Test set performance:')
    print("Weighted AUC(OvR):", weighted_auc_ovr)
    print("Weighted AUC(OvO):", weighted_auc_ovo)
    print(f'Weighted F1 score = {f1}')
    print(f'Weighted Precision = {precision}, Weighted Recall = {recall}, Accuracy = {accuracy}')
    print('Confusion matrix:\n')
    print(confusion_mat)

    # get the result of classification report
    report = classification_report(test_labels, test_predictions)
    print(f'Result of classification report:\n')
    print(report)
    return f1, weighted_auc_ovr, weighted_auc_ovo


In [6]:
kf = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=RANDOM_SEED)
val_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
#accuracy_list = []
weighted_f1_list = []
auc_ovr_list = []
auc_ovo_list = []
#precision_list = []
#recall_list = []
fold_id = 0
for train_index_, test_index in  kf.split(images, labels):
    train_images_fold_, test_images_fold = images[train_index_], images[test_index]
    train_labels_fold_, test_labels_fold = labels[train_index_], labels[test_index]
    for train_index, val_index in val_split.split(train_images_fold_, train_labels_fold_):
        train_images_fold, val_images_fold = train_images_fold_[train_index], train_images_fold_[val_index] 
        train_labels_fold, val_labels_fold = train_labels_fold_[train_index], train_labels_fold_[val_index]
         # train the model
        print(f'training on fold {fold_id}:')
        classifiers, alphas = boost_cnn(5, train_images_fold, train_labels_fold, val_X=val_images_fold, val_y=val_labels_fold)
        test_probabilities, test_predictions = boost_predict(test_images_fold, classifiers, alphas)
        f1, auc_ovr, auc_ovo = boost_test(test_labels_fold, test_probabilities, test_predictions)
        
        # store the values of metrics
        weighted_f1_list.append(f1)
        auc_ovr_list.append(auc_ovr)
        auc_ovo_list.append(auc_ovo)

        for idx, classifier in enumerate(classifiers):
            model_filename = os.path.join(model_weights_dir, f'boost_fold{fold_id}_No{idx}.h5')
            classifier.save(model_filename)     
        with open(os.path.join(model_weights_dir, 'info.txt'), 'w') as file:
            file.write(f'fold {fold_id}:\n')
            for idx, alpha in enumerate(alphas):
                file.write(f'Alpha No{idx}: {alpha}\n')
        fold_id += 1
        del classifiers, test_probabilities, test_predictions
        gc.collect()

training on fold 0:
No.0 iteration:
Epoch 1/50
1615/1615 [==============================] - 12s 6ms/step - loss: 3.5568e-05 - accuracy: 0.5607 - val_loss: 0.7973 - val_accuracy: 0.6195
Epoch 2/50
1615/1615 [==============================] - 9s 6ms/step - loss: 2.9444e-05 - accuracy: 0.6416 - val_loss: 0.7076 - val_accuracy: 0.6712
Epoch 3/50
1615/1615 [==============================] - 10s 6ms/step - loss: 2.5902e-05 - accuracy: 0.6928 - val_loss: 0.6397 - val_accuracy: 0.7113
Epoch 4/50
1615/1615 [==============================] - 10s 6ms/step - loss: 2.3260e-05 - accuracy: 0.7259 - val_loss: 0.5923 - val_accuracy: 0.7313
Epoch 5/50
1615/1615 [==============================] - 10s 6ms/step - loss: 2.0989e-05 - accuracy: 0.7559 - val_loss: 0.5426 - val_accuracy: 0.7611
Epoch 6/50
1615/1615 [==============================] - 12s 7ms/step - loss: 1.8804e-05 - accuracy: 0.7855 - val_loss: 0.5215 - val_accuracy: 0.7754
Epoch 7/50
1615/1615 [==============================] - 11s 7ms/step - 

In [7]:
print('Result of 5-fold Cross-validation on Test set data:')
print(f'weighted f1 score :{weighted_f1_list}\n')
print(f'Average of weighted auc OvR in 5 folds: {np.mean(auc_ovr_list)}')
print(f'Variance of weighted auc OvR in 5 folds: {np.var(auc_ovr_list)}')
print(f'Average of weighted auc OvO in 5 folds: {np.mean(auc_ovo_list)}')
print(f'Variance of weighted auc OvO in 5 folds: {np.var(auc_ovo_list)}')
print(f'Average of weighted F1 Score in 5 folds: {np.mean(weighted_f1_list)}')
print(f'Variance of Weighted F1 Score in 5 folds: {np.var(weighted_f1_list)}')

Result of 5-fold Cross-validation on Test set data:
weighted f1 score :[0.8752371209824367, 0.8727292729574977, 0.8623689826587951, 0.8641699837603957, 0.8651831349161622]

Average of weighted auc OvR in 5 folds: 0.9748342555653065
Variance of weighted auc OvR in 5 folds: 1.5583051821561223e-06
Average of weighted auc OvO in 5 folds: 0.972141682798646
Variance of weighted auc OvO in 5 folds: 2.0533872248601737e-06
Average of weighted F1 Score in 5 folds: 0.8679376990550575
Variance of Weighted F1 Score in 5 folds: 2.5806929075073585e-05
